In [1]:
#Import Dependencies
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

Using TensorFlow backend.
/usr/local/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#Import Dataset Helper Function
def get_data():
    #get train data
    train_data_path ='X1_train.csv'
    train = pd.read_csv(train_data_path)
    
    #get test data
    test_data_path ='X1_test.csv'
    test = pd.read_csv(test_data_path)
    
    return train , test

In [3]:
#Load train and test data into pandas DataFrames
train, test = get_data()

In [4]:
#Split Training Dataset
target = train.SW
train.drop(['SW', 'DEPTH'],axis = 1 , inplace = True)

In [5]:
#Split Testing Dataset
target_test = test.SW
test.drop(['SW', 'DEPTH'],axis = 1 , inplace = True)

In [ ]:


#Neural Network



In [6]:
#Neural Network Architecture

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1152      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 166,017
Trainable params: 166,017
Non-trainable params: 0
_________________________________________________________________


In [7]:
#Tensorboard Logging Settings

from keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1,
                          write_graph=True, write_images=False)

In [8]:
#Callback Function Settings

checkpoint_name = './Weights/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

callbacks_list = [es, checkpoint, tensorboard]


In [ ]:
#Training Neural Network

NN_model.fit(train, target, epochs=1000, batch_size=32, verbose = 0, validation_split = 0.2, callbacks=callbacks_list)



In [70]:
# Load weights file of the best model :
weights_file = 'Weights/Weights-165--0.00469.hdf5' # choose the best checkpoint 
NN_model.load_weights(weights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])



In [ ]:


#SUPPORT VECTOR REGRESSION



In [10]:
#SUPPORT VECTOR REGRESSION ie SVM for regression
from sklearn.svm import SVR
model = SVR()

model.fit(train,target)



SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [12]:
# save the model to disk
import pickle

filename = 'svr_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [13]:
# load the model from disk
model = pickle.load(open(filename, 'rb'))


In [ ]:


#Comparing The Performance of Neural Network to SVR



In [14]:
#SVR Prediction

predict = model.predict(test)
MAE = mean_absolute_error(target_test , predict)
print('SVR validation MAE = ', MAE)


SVR validation MAE =  0.11791212597257839


In [15]:
#Neural Network Prediction

predictions = NN_model.predict(test)
MAE_2 = mean_absolute_error(target_test , predictions)
print('ANN validation MAE = ', MAE_2)

ANN validation MAE =  0.005824961309607124


In [ ]:


#Neural Network & SVR Predictions With Respect To Actual SW Data
#For Comparison



In [37]:
#Load Dataset
X1_Log = pd.read_csv('X1_Log_Pro.csv')

In [39]:
#Split Dataset
actual = X1_Log.SW
depth = X1_Log.DEPTH
X1_Log.drop(['SW', 'DEPTH'],axis = 1 , inplace = True)

In [40]:
#Neural Network Prediction
X1_Log_NN_pred = NN_model.predict(X1_Log)

In [41]:
#SVR Prediction
X1_Log_SVR_pred = model.predict(X1_Log)

In [ ]:


#Creating Dataset of Neural Network & SVR Predictions With Respect To Actual SW Data



In [42]:
df = pd.DataFrame(depth)
df

,DEPTH
0,10000.0
1,10000.5
2,10001.0
3,10001.5
4,10002.0
5,10002.5
6,10003.0
7,10003.5
8,10004.0
9,10004.5


In [43]:
df['ANN'] = X1_Log_NN_pred

In [44]:
df['SVR'] = X1_Log_SVR_pred

In [45]:
df['ACTUAL'] = actual

In [46]:
df

,DEPTH,ANN,SVR,ACTUAL
0,10000.0,1.000050,0.900472,1.0
1,10000.5,0.999969,0.916985,1.0
2,10001.0,0.999959,0.910308,1.0
3,10001.5,0.999988,0.917851,1.0
4,10002.0,1.000188,0.922324,1.0
5,10002.5,1.000322,0.913891,1.0
6,10003.0,1.000170,0.901091,1.0
7,10003.5,1.000057,0.904266,1.0
8,10004.0,0.999983,0.901952,1.0
9,10004.5,0.999809,0.910147,1.0


In [48]:
#Save Dataset
df.to_csv("result.csv", encoding='utf-8', index=False)

In [ ]:


#Visualisations of Neural Network & SVR Predictions With Respect To Actual SW Data



In [52]:
#Change Username & Key According To Plotly Account

import plotly 
plotly.tools.set_credentials_file(username='username', api_key='key')

In [53]:
from plotly.plotly import iplot

# import graph objects as "go"
import plotly.graph_objs as go

In [68]:
#Plotting ANN With Respect To Actual SW

# Creating trace1
trace1 = go.Scatter(
                    x = df.ANN,
                    y = df.DEPTH,
                    mode = "lines",
                    name = "ANN",
                    marker = dict(color = 'rgba(128, 191, 255, 0.8)'),
                    text= df.ANN)
# Creating trace2
trace2 = go.Scatter(
                    x = df.ACTUAL,
                    y = df.DEPTH,
                    mode = "lines",
                    name = "ACTUAL",
                    marker = dict(color = 'rgba(80, 26, 80, 0.8)'),
                    text= df.ACTUAL)
data = [trace1, trace2]
layout = dict(title = 'Water Saturation Prediction With Respect To Depth',
              xaxis= dict(title= 'Water Saturation',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Depth ft',ticklen= 5,zeroline= False),
              autosize=False,
              width=500,
              height=1000
             )
fig = dict(data = data, layout = layout)
iplot(fig)

In [69]:
#Plotting SVR With Respect To Actual SW

# Creating trace1
trace1 = go.Scatter(
                    x = df.SVR,
                    y = df.DEPTH,
                    mode = "lines",
                    name = "SVR",
                    marker = dict(color = 'rgba(255, 102, 153, 0.8)'),
                    text= df.SVR)
# Creating trace2
trace2 = go.Scatter(
                    x = df.ACTUAL,
                    y = df.DEPTH,
                    mode = "lines",
                    name = "ACTUAL",
                    marker = dict(color = 'rgba(80, 26, 80, 0.8)'),
                    text= df.ACTUAL)
data = [trace1, trace2]
layout = dict(title = 'Water Saturation Prediction With Respect To Depth',
              xaxis= dict(title= 'Water Saturation',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Depth ft',ticklen= 5,zeroline= False),
              autosize=False,
              width=500,
              height=1000
             )
fig = dict(data = data, layout = layout)
iplot(fig)